## 1. Instalamos dependencias y librerías

In [ ]:
%pip install -U -q langchain>=0.2.5 langchain-community>=0.2.0 langchain-text-splitters>=0.2.0 langchain-google-genai>=0.0.10 chromadb>=0.5.0 tiktoken>=0.7.0 pypdf>=4 python-dotenv>=1.0.1

## 2. Importamos las librerías necesarias

In [ ]:
import os
from pathlib import Path
from typing import List

from dotenv import load_dotenv
load_dotenv()  # Carga variables de entorno (GOOGLE_API_KEY, etc.)

# LangChain loaders, splitters, vectorstore, LLM/embeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

# Embeddings y modelo de chat con Google Gemini
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

# Utilidad
from tqdm import tqdm

## 3. Configuramos el entorno

In [ ]:
# 🛠️ Configuración
PDF_DIR = Path("./docs")             # <- Carpeta con PDFs
PERSIST_DIR = Path("./chroma_pdfs_gemini")  # Donde se guardará Chroma
PERSIST_DIR.mkdir(parents=True, exist_ok=True)

# <FILL_IN> Completa estos valores
CHUNK_SIZE = <FILL_IN>  # Tamaño de cada chunk (trozo) de texto
CHUNK_OVERLAP = <FILL_IN>  # Solapamiento entre chunks
TOP_K = 4  # Número de documentos recuperados

# Modelos de Gemini
EMBEDDING_MODEL = "<FILL_IN>"  # Modelo de embeddings (ej: "models/text-embedding-004")
CHAT_MODEL = "<FILL_IN>"  # Modelo de chat (ej: "gemini-2.5-flash")

# Verificar clave
assert os.getenv("GOOGLE_API_KEY"), "Falta GOOGLE_API_KEY en variables de entorno o .env"
print(f"📁 Carpeta PDFs: {PDF_DIR.resolve()}")
print(f"🗂️ Persistencia Chroma: {PERSIST_DIR.resolve()}")

## 4. Cargamos los PDFs

In [ ]:
def load_pdfs_from_dir(directory: Path, recursive: bool = True):
    """Carga todos los PDFs de una carpeta de forma recursiva."""
    pattern = "**/*.pdf" if recursive else "*.pdf"
    pdf_paths = sorted([p for p in directory.glob(pattern) if p.is_file()])
    all_docs = []
    for pdf in tqdm(pdf_paths, desc="Cargando PDFs"):
        try:
            docs = PyPDFLoader(str(pdf)).load()
            # Añadimos metadatos útiles
            for d in docs:
                d.metadata = d.metadata or {}
                d.metadata["source"] = str(pdf.resolve())
            all_docs.extend(docs)
        except Exception as e:
            print(f"⚠️ Error leyendo {pdf}: {e}")
    print(f"📚 Documentos (páginas) cargados: {len(all_docs)}")
    return all_docs

raw_docs = load_pdfs_from_dir(PDF_DIR, recursive=True)

## 5. Chunking (Trozamos los documentos)

In [ ]:
# <FILL_IN> Crea el splitter con los parámetros configurados arriba
splitter = RecursiveCharacterTextSplitter(
    chunk_size=<FILL_IN>,
    chunk_overlap=<FILL_IN>,
    separators=["\n\n", "\n", " ", ""],
)
chunks = splitter.split_documents(raw_docs)
print(f"✂️ Chunks generados: {len(chunks)}")
print(f"📝 Ejemplo de primer chunk:\n{chunks[0].page_content[:200]}...")

## 6. Crear Embeddings y Vector Store

In [ ]:
# <FILL_IN> Crea los embeddings con Gemini
embeddings = GoogleGenerativeAIEmbeddings(model=<FILL_IN>)

# <FILL_IN> Crea el vector store con Chroma
vectorstore = Chroma.from_documents(
    documents=<FILL_IN>,
    embedding=<FILL_IN>,
    persist_directory=str(PERSIST_DIR),
)

# <FILL_IN> Crea el retriever
retriever = <FILL_IN>
print("✅ Chroma persistido y retriever creado")

## 7. Creamos la Tool (Retriever) para el agente

In [ ]:
from langchain_core.tools.retriever import create_retriever_tool

# <FILL_IN> Crea la retriever tool
retriever_tool = create_retriever_tool(
    <FILL_IN>,  # retriever
    "<FILL_IN>",  # nombre de la tool
    "<FILL_IN>",  # descripción
)

print("✅ Retriever tool creada")

### 7.1 Test de la tool

In [ ]:
# <FILL_IN> Prueba la tool con una pregunta de ejemplo
resultado = retriever_tool.invoke({"query": "<FILL_IN>"})
print(resultado)

## 8. Nodo: Genera query o responde directamente

In [ ]:
from langgraph.graph import MessagesState

# <FILL_IN> Instancia el modelo de chat
response_model = ChatGoogleGenerativeAI(model=<FILL_IN>, temperature=<FILL_IN>)

def genera_query_o_responde(state: MessagesState):
    """Decide si recuperar información o responder directamente."""
    # <FILL_IN> Usa bind_tools para conectar la retriever_tool
    response = (
        response_model
        .bind_tools([<FILL_IN>]).invoke(state[<FILL_IN>])
    )
    return {"messages": [response]}

### 8.1 Test: Pregunta que no necesita búsqueda

In [ ]:
# <FILL_IN> Prueba con una pregunta que no necesita búsqueda
input_test = {"messages": [{"role": "user", "content": "<FILL_IN>"}]}
respuesta = genera_query_o_responde(input_test)
respuesta["messages"][-1].pretty_print()

### 8.2 Test: Pregunta que requiere búsqueda semántica

In [ ]:
# <FILL_IN> Prueba con una pregunta que requiere búsqueda
input_test = {
    "messages": [
        {
            "role": "user",
            "content": "<FILL_IN>",  # Una pregunta sobre tus PDFs
        }
    ]
}
respuesta = genera_query_o_responde(input_test)
respuesta["messages"][-1].pretty_print()

## 9. Nodo: Evaluar relevancia de documentos

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal

GRADE_PROMPT = (
    "Eres un evaluador que determina la relevancia de un documento recuperado respecto a una pregunta del usuario. \n "
    "Aquí tienes el documento recuperado: \n\n {context} \n\n"
    "Aquí tienes la pregunta del usuario: {question} \n"
    "Si el documento contiene palabra(s) clave o significado semántico relacionado con la pregunta del usuario, califícalo como relevante. \n"
    "Da una puntuación binaria 'si' o 'no' para indicar si el documento es relevante para la pregunta."
)


class GradeDocuments(BaseModel):
    """Califica los documentos utilizando una puntuación binaria"""
    binary_score: str = Field(description="Puntuación: 'si' si es relevante, o 'no' si no lo es")


# <FILL_IN> Instancia el modelo grader
grader_model = ChatGoogleGenerativeAI(model=<FILL_IN>, temperature=<FILL_IN>)


def grade_documents(state: MessagesState) -> Literal["genera_respuesta", "rescribir_question"]:
    """Determina si los documentos recuperados son relevantes."""
    print("⏳ Evaluando relevancia de documentos...")
    # <FILL_IN> Extrae la pregunta y el contexto
    question = <FILL_IN>
    context = <FILL_IN>
    
    prompt = GRADE_PROMPT.format(question=question, context=context)
    response = (
        grader_model
        .with_structured_output(GradeDocuments).invoke(
            [{"role": "user", "content": prompt}]
        )
    )
    score = response.binary_score
    print(f"📊 Score de relevancia: {score}")
    
    if score == "si":
        return "genera_respuesta"
    else:
        return "rescribir_question"

## 10. Nodo: Rescribir la pregunta si no es clara

In [ ]:
REWRITE_PROMPT = (
    "Analiza detenidamente la siguiente pregunta e intenta comprender la intención o el significado profundo que transmite.\n"
    "Pregunta original:\n ------- \n{question}\n ------- \n"
    "Ahora, reescribe la pregunta para que sea más clara, precisa y fácil de entender:"
)


def rescribir_question(state: MessagesState):
    """Reescribe la pregunta del usuario para mejorarla."""
    print("✏️ Reescribiendo pregunta...")
    messages = state["messages"]
    # <FILL_IN> Extrae la pregunta original
    question = <FILL_IN>
    prompt = REWRITE_PROMPT.format(question=question)
    response = response_model.invoke([{"role": "user", "content": prompt}])
    return {"messages": [{"role": "user", "content": response.content}]}

## 11. Nodo: Generar la respuesta final

In [ ]:
GENERATE_PROMPT = (
    "Eres un asistente para tareas de preguntas y respuestas. "
    "Utiliza los siguientes fragmentos de contexto recuperado para responder a la pregunta. "
    "Si no sabes la respuesta, simplemente indica que no la sabes. "
    "Utiliza un máximo de tres frases y mantén la respuesta concisa.\n"
    "Pregunta: {question} \n"
    "Contexto: {context}"
)


def genera_respuesta(state: MessagesState):
    """Genera la respuesta final basada en el contexto."""
    print("🤖 Generando respuesta...")
    # <FILL_IN> Extrae la pregunta y el contexto
    question = <FILL_IN>
    context = <FILL_IN>
    prompt = GENERATE_PROMPT.format(question=question, context=context)
    response = response_model.invoke([{"role": "user", "content": prompt}])
    return {"messages": [response]}

## 12. Construir el grafo (workflow)

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

# <FILL_IN> Crea el StateGraph
workflow = StateGraph(<FILL_IN>)

# <FILL_IN> Añade los nodos
workflow.add_node(<FILL_IN>)  # genera_query_o_responde
workflow.add_node("retrieve", ToolNode([<FILL_IN>]))
workflow.add_node(<FILL_IN>)  # rescribir_question
workflow.add_node(<FILL_IN>)  # genera_respuesta

# <FILL_IN> Añade las aristas
workflow.add_edge(<FILL_IN>, "genera_query_o_responde")  # START a genera_query_o_responde

workflow.add_conditional_edges(
    "genera_query_o_responde",
    tools_condition,
    {
        "tools": "retrieve",
        END: END,
    },
)

workflow.add_conditional_edges(
    "retrieve",
    <FILL_IN>,  # grade_documents
)

workflow.add_edge("genera_respuesta", <FILL_IN>)  # END
workflow.add_edge("rescribir_question", "genera_query_o_responde")

print("✅ Grafo construido")

## 13. Compilar el grafo

In [ ]:
# <FILL_IN> Compila el grafo
graph = <FILL_IN>
print("✅ Grafo compilado")

## 14. Visualizar el grafo

In [ ]:
from IPython.display import Image, display

# <FILL_IN> Visualiza el grafo
display(Image(<FILL_IN>))

## 15. Ejecutar el grafo - Ejemplo 1

In [ ]:
from pprint import pprint

# <FILL_IN> Define tu pregunta
pregunta = "<FILL_IN>"

# Ejecuta el grafo
for chunk in graph.stream({"messages": [{"role": "user", "content": pregunta}]}):
    for node, update in chunk.items():
        print(f"📘 Update from node: {node}")
        print("-" * 40)
        messages = update.get("messages", [])
        last_msg = messages[-1]
        try:
            if hasattr(last_msg, "content"):
                print("📝 Contenido:")
                print(last_msg.content)
            else:
                pprint(last_msg)
        except Exception as e:
            print(f"❌ Error: {str(e)}")
        print("-" * 40 + "\n")

## 16. Ejecutar el grafo - Resultado Final

In [ ]:
from IPython.display import display, Markdown

# <FILL_IN> Define tu pregunta
pregunta = "<FILL_IN>"

# Ejecuta el grafo y muestra solo la respuesta final
resultado = graph.invoke({"messages": [{"role": "user", "content": pregunta}]})
display(Markdown(resultado["messages"][-1].content))